1. OriginalDogNet: Score: 0.60180
2. + Normalization_and_slight_contrast_stretch. Score: 0.63 (worse)
3. + larger resolution(150 x 150):0.62
4. - Normalization_and_slight_contrast_stretch and retain w/h ratio 

reference material: https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html and VGG-16

In [135]:
import os,random
import numpy as np
import pandas as pd

import PIL
from PIL import Image

import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns
%matplotlib inline

from keras.models import Sequential
from keras.layers import Input,Dropout,Flatten,Convolution2D,MaxPooling2D,Dense,Activation
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint,Callback,EarlyStopping
from keras.utils import np_utils

Preparing the Data_1

1. normalize the luminance (brightness) values to have mean 0 and standard deviation 1
2. apply a slight contrast stretch


In [136]:
def norm_image(img):
    """
    Normalize PIL image
    Normalize luminace to (mean,std) = (0,1),and applies a [1%,99%] contrast stretch
    """
    #convert to YCbCr
    img_y,img_b,img_r = img.convert('YCbCr').split()
    img_y_np = np.asarray(img_y).astype(float)
    
    #Normalize
    img_y_np /= 255
    img_y_np -= img_y_np.mean()
    img_y_np /= img_y_np.std()
    
    #contrast stretch
    scale = np.max([np.abs(np.percentile(img_y_np,1.0)),np.abs(np.percentile(img_y_np,99.0))])
    img_y_np = img_y_np/scale
    img_y_np = np.clip(img_y_np,-1.0,1.0)
    img_y_np = (img_y_np+1.0)/2.0
    
    img_y_np = (img_y_np * 255 + 0.5).astype(np.uint8)
    img_y = Image.fromarray(img_y_np)
    img_ybr = Image.merge('YCbCr',(img_y,img_b,img_r))
    
    #convert back to RGB
    img_nrm = img_ybr.convert('RGB')
    
    return img_nrm

Preparing the Data_2

1. resize the images to 64x64
2. sample 2000 images(8%) of the data to run efficiently
3. separate cats and dogs for exploratory analysis

In [137]:
ROWS = 150
COLS = 150
SIZE = 150

BASE_DIR = 'F:/Data Science/Dataset/DogVsCat/'
BASE_OUT = os.path.join(BASE_DIR,'data{}/'.format(SIZE))
TRAIN_DIR = os.path.join(BASE_OUT,'train/')
TEST_DIR = os.path.join(BASE_OUT,'test/')

#TRAIN_DIR = 'F:/Data Science/Dataset/DogVsCat/train/'
#TEST_DIR = 'F:/Data Science/Dataset/DogVsCat/test/'

MODEL_WEIGHT_DIR = 'F:/Data Science/Kaggle/kaggle_sourcecode/DogsvsCats/'


CHANNELS = 3

train_images = [TRAIN_DIR + i for i in os.listdir(TRAIN_DIR)]#use this for full datase
train_dogs = [TRAIN_DIR + i for i in os.listdir(TRAIN_DIR) if 'dog' in i]
train_cats = [TRAIN_DIR + i for i in os.listdir(TRAIN_DIR) if 'cat' in i]

test_images = [TEST_DIR + i for i in os.listdir(TEST_DIR)]

#slice datasets for memory efficiency , delete if using full dataset
#train_images = train_dogs[:2000] + train_cats[:2000]
#train_images = train_dogs[:1]
random.shuffle(train_images)
#test_images = test_images[:25]

def read_image(file_path):
    img = Image.open(file_path)
    #img_nrm = norm_image(img)
    #img_res = img_nrm.resize((ROWS,COLS),resample = PIL.Image.BICUBIC)
    #return img_res
    return img

def prep_data(images):
    count = len(images)
    data = np.ndarray((count,CHANNELS,ROWS,COLS),dtype = np.uint8)
    
    for i, image_file in enumerate(images):
        image = read_image(image_file)   
        img_np = np.asarray(image).astype(np.uint8)
        #print (img_np)
        data[i] = img_np.T
        #print (img_np.T)
        #print (i)
        if i%5000 == 0:
            print('Processed {} of {}'.format(i,count))
    return data

train=prep_data(train_images)
test=prep_data(test_images)

print("Train shape: {}".format(train.shape))
print("Test shape : {}".format(test.shape))   

Processed 0 of 25000
Processed 5000 of 25000
Processed 10000 of 25000
Processed 15000 of 25000
Processed 20000 of 25000
Processed 0 of 12500
Processed 5000 of 12500
Processed 10000 of 12500
Train shape: (25000, 3, 150, 150)
Test shape : (12500, 3, 150, 150)


Generating the Labels

We're dealing with a binary classification problem here - (1) dog (0) cat.
The lables is obtained by looping over the file names in the train directory 

In [138]:
labels = []
for i in train_images:
    if 'dog' in i:
        labels.append(1)
    else:
        labels.append(0)

#sns.countplot(labels)
#sns.plt.title('Cats and Dogs')

Checking out Cats and Dogs

A quick side-by-side comparison of the animals

In [139]:
#def show_cats_and_dogs(idx):
#    cat = read_image(train_cats[idx])
#    dog = read_image(train_dogs[idx])
#    
#    plt.figure(figsize = (8,4))
#    plt.subplot(121)
#    plt.title('Cat')
#    plt.imshow(cat)
#    
#    plt.subplot(122)
#    plt.title('Dog')
#    plt.imshow(dog)
#    
#    plt.tight_layout()
#    plt.show()

#for idx in range(0,5):
#    show_cats_and_dogs(idx)

Average Cat and Dog Photo

for fun

In [140]:
#dog_avg = np.array([dog[0].T for i,dog in enumerate(train) if labels[i] == 1]).mean(axis = 0)
#plt.imshow(dog_avg)
#plt.title('Your Average Dog')

In [141]:
#cat_avg = np.array([cat[0].T for i ,cat in enumerate(train) if labels[i] == 0]).mean(axis = 0)
#plt.imshow(cat_avg)
#plt.title('Your Average Cat')

CatDogNet-16
*  based on VGG-16: very deep convolutional Networks for Large-Scale Visual Recognition

A scaled down version of the VGG-16,with a few notable changes
1. Number of convolution filters cut in half, (64->32),(128->64),(256->128),(512->256)
2. fully connected(Dense) layers scaled down (FC-4096,FC-4096,FC-1000)->(FC-256,FC-256,FC-1)
3. Optimizer changed to RMSprop
4. Output layer activation set to sigmoid for binary crossentropy
5. Some Layes commemted out for efficiency

In [ ]:
optimizer = RMSprop(lr = 1e-4)
objective = 'binary_crossentropy'

def catdog():
    model = Sequential()
    
    model.add(Convolution2D(32,3,3,border_mode = 'same',input_shape = (3,ROWS,COLS),activation = 'relu'))
    model.add(Convolution2D(32,3,3,border_mode = 'same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2),dim_ordering = 'th'))
    
    model.add(Convolution2D(64,3,3,border_mode = 'same',activation = 'relu'))
    model.add(Convolution2D(64,3,3,border_mode = 'same',activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),dim_ordering = 'th'))
    
    model.add(Convolution2D(128,3,3,border_mode = 'same',activation = 'relu'))
    model.add(Convolution2D(128,3,3,border_mode = 'same',activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2,2),dim_ordering = 'th'))
    
    model.add(Convolution2D(256,3,3,border_mode = 'same',activation = 'relu'))
    model.add(Convolution2D(256,3,3,border_mode = 'same',activation = 'relu'))
    #model.add(Convolution2D(256,3,3,border_model = 'same',activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2,2),dim_ordering = 'th'))
    
    # model.add(Convolution2D(256, 3, 3, border_mode='same', activation='relu'))
    # model.add(Convolution2D(256, 3, 3, border_mode='same', activation='relu'))
    # model.add(Convolution2D(256, 3, 3, border_mode='same', activation='relu'))
    # model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(256,activation = 'relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(256,activation = 'relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    model.compile(loss = objective,optimizer = optimizer,metrics=['accuracy'])
    
    return model

model = catdog()    

Train and Predict

1. use Keras's early stopping callback to end training when the validation loss stop improving, otherwise the model will overfit
2. tracking the loss history on each epoch to visualize the overfitting trend
3. A slice of 1000 images was used to fit the model for CPU efficency. The model's performance improves significantly when used on the entire dataset

In [ ]:
nb_epoch = 100
batch_size = 32

##Callback for loss logging per epoch
class LossHistory(Callback):
    def on_train_begin(self,logs = {}):
        self.losses = []
        self.val_losses = []
    
    def on_epoch_end(self,batch,logs = {}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        
early_stopping = EarlyStopping(monitor = 'val_loss',patience = 3,verbose = 1,mode = 'auto')

def run_catdog():
    history = LossHistory()
    model.fit(train,labels,batch_size = batch_size,nb_epoch = nb_epoch,
              validation_split = 0.25,verbose = 0,shuffle = True,callbacks = [history,early_stopping])
    predictions = model.predict(test,verbose = 0)
    return predictions,history

predictions,history = run_catdog()

In [ ]:
loss = history.losses
val_loss = history.val_losses

plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('VGG-16 Loss Trend')
plt.plot(loss,'blue',label = 'Training Loss')
plt.plot(val_loss,'green',label = 'Validation Loss')
plt.xticks(range(0,nb_epoch)[0::2])
plt.legend()
plt.show()

1. what's the confident is the model
2. Run on the full dataset with a GPU 

In [ ]:
for i in range(0,10):
    if predictions[i,0]>=0.5:
        print('I am {:.2%} sure this is a Dog'.format(predictions[i][0]))
    else:
        print('I am {:.2%}sure this is a Cat'.format(1-predictions[i][0]))
    
    plt.imshow(test[i].T)
    plt.show()

Save model

In [ ]:
from keras.models import model_from_json

json_string = model.to_json()

import json
models_name = 'VGG_16_loss_trend.txt'
model_path = os.path.join(MODEL_WEIGHT_DIR,models_name)
with open(model_path,'w') as outfile:
    json.dump(json_string,outfile)
#model =  model_from_json(json_string)

Save weights

In [ ]:
weights_name = 'VGG_16_loss_trend_full_weights.txt'
weight_path = os.path.join(MODEL_WEIGHT_DIR,weights_name)
model.save_weights(weight_path)

#model.load_weights(weight_path,by_name = False)

In [ ]:
#print ('Processing Submission File...')
def num_key(file):
    return int(file.split('.')[0])
submission = pd.DataFrame()
submission['id'] = [num_key(i) for i in os.listdir(TEST_DIR)]
submission['label'] = predictions
submission.set_index(['id'],inplace = True,drop = True)
submission.to_csv('catvsdog.csv')
#predictions